In [ ]:
%pip install opencv-python matplotlib kagglehub python-dotenv jupyter ipykernel

In [ ]:
%pip uninstall -y numpy
%pip install "numpy>=1.23.5,<2"
%pip install tensorflow-macos tensorflow-metal tensorflow-hub

Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import cv2
import os
import kagglehub
import numpy as np
import matplotlib.pyplot as plt
import random

Clearing cache

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()

import gc
gc.collect()

GPU Check

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print("Using gpu: ", gpus)
else:
    print("no gpu found using cpu")

Config

In [ ]:
COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
    'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
    'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
IMG_SIZE = 320
GRID_SIZE = 10
NUM_CLASSES = 3
BATCH_SIZE = 8
EPOCHS = 50
CONF_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

## Dataset

Loading the dataset

In [ ]:
path = kagglehub.dataset_download("ultralytics/coco128")

print(path)
print(os.listdir(path))

In [ ]:
IMG_DIR = os.path.join(path, "coco128","images", "train2017")
LABEL_DIR = os.path.join(path, "coco128","labels","train2017")

In [ ]:
print("images: ",len(os.listdir(IMG_DIR)))
print("labels: ",len(os.listdir(LABEL_DIR)))

Loading the image and label in sorted pairs

In [ ]:
image_files = sorted(os.listdir(IMG_DIR))
label_files = sorted(os.listdir(LABEL_DIR))

Label Encoding

In [ ]:
def encode_labels(boxes, labels):
    target = np.zeroes((GRID_SIZE, GRID_SIZE, 5+NUM_CLASSES), dtype=np.float32)

    for box, clss in zip(boxes,labels):
        cx, cy, w, h = box

        # grid cell
        grid_x = int(cx*GRID_SIZE)
        grid_y = int(cy*GRID_SIZE)

        # cell offset
        dx = cx*GRID_SIZE - grid_x
        dy = cy*GRID_SIZE - grid_y




Model Architecture

Loss Function

Training loops